In [2]:
import random
from datasets import load_dataset

In [4]:
ds = load_dataset("clarin-knext/fiqa-pl", "corpus")

random.seed(1234)
ds_t1000 = []

for obj in random.sample(list(ds['corpus']),1000):
    ds_t1000.append(obj['text'])

ds_t1000[:3]

['Wiele osób posiadających Tesle posiada również panele słoneczne i może ładować swój samochód energią słoneczną. Na razie nasza sieć energetyczna nadal opiera się na archaicznych technologiach wytwarzania energii, ale Tesla i inni producenci samochodów dążą do dodania do naszych domów i sieci większej ilości odnawialnych źródeł energii.',
 'Byłem zarówno klientem ich, jak i NetBanku. Kiedy ING kupił NetBank, czułem, że bardzo słabo radzili sobie z przejściem z punktu widzenia obsługi klienta. Czułem, że tak naprawdę nie cenią mnie jako klienta i nie zrobili nic, aby ułatwić mi przejście. Z tego powodu postanowiłem wyciągnąć pieniądze ze wszystkich kont, które miałem przy nich, i nigdy nie oglądałem się za siebie. Otrzymuję teraz znacznie lepsze stawki dzięki wysokowydajnemu kontu czekowemu z nagrodami w moim lokalnym banku. Nadal mogę płacić rachunki online lub wysyłać czeki do lokalnego banku, jeśli nie chcę fizycznie iść do oddziału lub bankomatu.',
 'lol @ konta oszczędnościowe. Ni

In [5]:
from ollama import chat
from ollama import ChatResponse

# Testowe zapytanie modelu

In [50]:
# Note: zmienić przy zmianie modelu
model_name = 'SpeakLeash/bielik-7b-instruct-v0.1-gguf'

In [ ]:
response: ChatResponse = chat(model=model_name, messages=[
    {
        'role': 'user',
        'content': 'Why is the sky green?',
    },
])

print(response.message.content)

 The sky appears green due to a process called Rayleigh scattering. This occurs when sunlight interacts with molecules in the Earth's atmosphere, particularly oxygen and nitrogen. These molecules are much smaller than the wavelength of visible light (which is around 500 nanometers), so they can only scatter certain colors or "wavelengths" of light. Blue light is scattered more efficiently than other colors, which is why the sky often appears blue during the day.

However, when the sun is low on the horizon, sunlight has to travel through more of the Earth's atmosphere before reaching our eyes. This means that blue light is scattered more and less green light reaches our eyes, making the sky appear greenish or bluish during sunrise and sunset. The intensity of green color can vary depending on the angle of the sun and other factors like weather conditions.


In [47]:
prompt = "Odczytaj wszystkie imiona osób, nazwy organizacji, państw i miejsc z podanego tekstu. Następnie wypisz znalezione słowa, jeżeli wystąpiły w tekście, dzieląc je na odpowiednie kategorie. Skup się na ty, aby faktycznie znalezione wyrazy to były imiona lub nazwy. Postaraj, aby imiona osób, to faktyczne imiona, a nie określenia ludzi. Najważniejsze jest, abyś wypisał nazwy, które wystąpiły BEZPOŚREDNIO w tekście. Oto dany tekst: "

In [ ]:
response: ChatResponse = chat(model=model_name, messages=[
    {
        'role': 'user',
        'content': prompt + ds_t1000[0],
    },
])

print(ds_t1000[0])
print(response.message.content)

Wiele osób posiadających Tesle posiada również panele słoneczne i może ładować swój samochód energią słoneczną. Na razie nasza sieć energetyczna nadal opiera się na archaicznych technologiach wytwarzania energii, ale Tesla i inni producenci samochodów dążą do dodania do naszych domów i sieci większej ilości odnawialnych źródeł energii.
 Imiona osób:
- Tesla
Nazwy organizacji:
- Solar Panels
Nazwy państw:
- USA (United States of America)
Nazwy miejsc:
- Home
